# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself. Everyone will get slightly different results depending on their machine.

Writing performant code is extremely important in scientific computing, but also can often be quite mysterious and confusing.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
    - C implementations (optional, require gcc)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [8]:
# for i=2:5
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)
# end


10000000-element Vector{Float64}:
 0.5459898619397224
 0.8613700731902164
 0.5266105641904599
 0.14451262584379554
 0.5814116312953571
 0.9689221449791242
 0.16541831218476333
 0.6177443510069358
 0.07469520851098321
 0.3698074741552666
 ⋮
 0.29225740128783173
 0.8828072550838441
 0.31904786744008173
 0.41033007953251077
 0.7800124377383492
 0.7437791188318861
 0.018303232938248537
 0.7423270776141624
 0.009259510979367058

In [9]:
sum(a)

4.999269608768198e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [10]:
@time sum(a)

  0.003186 seconds (1 allocation: 16 bytes)


4.999269608768198e6

In [11]:
@time sum(a)

  0.002764 seconds (1 allocation: 16 bytes)


4.999269608768198e6

In [12]:
@time sum(a)

  0.002790 seconds (1 allocation: 16 bytes)


4.999269608768198e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [3]:
using Pkg
Pkg.add("BenchmarkTools")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
┌ Warning: The active manifest file at `/home/snowztail/Repositories/Introduction-to-Julia/Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.7/Pkg/src/manifest.jl:287
  No Changes to `~/Repositories/Introduction-to-Julia/Project.toml`
  No Changes to `~/Repositories/Introduction-to-Julia/Manifest.toml`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /home/snowztail/Repositories/Introduction-to-Julia/Manifest.toml:0


In [4]:
using BenchmarkTools  

# 1. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [5]:
using Pkg; Pkg.add("PyCall")
using PyCall

   Resolving package versions...
  No Changes to `~/Repositories/Introduction-to-Julia/Project.toml`
  No Changes to `~/Repositories/Introduction-to-Julia/Manifest.toml`


In [6]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [13]:
pysum(a)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/snowztail/.julia/packages/PyCall/BD546/src/numpy.jl:67


4.999269608767693e6

In [14]:
pysum(a) ≈ sum(a)

true

In [15]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  6
  --------------
  minimum time:     531.623 ms (0.00% GC)
  median time:      532.189 ms (0.00% GC)
  mean time:        534.420 ms (0.00% GC)
  maximum time:     542.764 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1

In [16]:
d = Dict()  # a "dictionary", i.e. an associative array
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Python built-in" => 531.623

In [17]:
using Plots

In [18]:
using Statistics # bring in statistical support for standard deviations
t = py_list_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

MethodError: MethodError: no method matching floatrange(::Type{Float64}, ::Int64, ::Int64, ::Float64, ::Int64)
Closest candidates are:
  floatrange(::Type{T}, ::Integer, ::Integer, !Matched::Integer, ::Integer) where T at /usr/share/julia/base/twiceprecision.jl:370
  floatrange(!Matched::AbstractFloat, !Matched::AbstractFloat, ::Real, ::AbstractFloat) at /usr/share/julia/base/twiceprecision.jl:384

# 2. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [19]:
using Pkg; Pkg.add("Conda")
using Conda

   Resolving package versions...
  No Changes to `~/Repositories/Introduction-to-Julia/Project.toml`
  No Changes to `~/Repositories/Introduction-to-Julia/Manifest.toml`


In [20]:
Conda.add("numpy")
numpy_sum = pyimport("numpy")["sum"]


PREFIX=/home/snowztail/.julia/conda/3
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/snowztail/.julia/conda/3

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py39h06a4308_2
    - cffi==1.15.0=py39hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - colorama==0.4.4=pyhd3eb1b0_0
    - conda-content-trust==0.1.1=pyhd3eb1b0_0
    - conda-package-handling==1.8.1=py39h7f8727e_0
    - conda==4.12.0=py39h06a4308_0
    - cryptography==36.0.0=py39h9ce1e76_0
    - idna==3.3=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.3=h7f8727e_2
    - openssl==1.1.1n=h7f8727e_0
    - pip==21.2.4=py39h06a4308_0
    - pycosat==0.6.3=py

mkl-2021.4.0         | 142.6 MB  | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


┌ Info: Downloading miniconda installer ...
└ @ Conda /home/snowztail/.julia/packages/Conda/sNGum/src/Conda.jl:192
┌ Info: Installing miniconda ...
└ @ Conda /home/snowztail/.julia/packages/Conda/sNGum/src/Conda.jl:202
┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /home/snowztail/.julia/packages/Conda/sNGum/src/Conda.jl:128


PyObject <function sum at 0x7fe03dfb64d0>

In [21]:
numpy_sum(a)

4.9992696087682005e6

In [22]:
numpy_sum(a) ≈ sum(a)# type \approx and then <TAB> to get the ≈ symbolb

true

In [23]:
≈  # alias for the `isapprox` function

isapprox (generic function with 18 methods)

In [24]:
?isapprox

ErrorException: syntax: invalid identifier name "?"

In [25]:
py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  6
  --------------
  minimum time:     2.730 ms (0.00% GC)
  median time:      2.928 ms (0.00% GC)
  mean time:        2.959 ms (0.00% GC)
  maximum time:     4.337 ms (0.00% GC)
  --------------
  samples:          1685
  evals/sample:     1

In [26]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Python numpy"    => 2.72977
  "Python built-in" => 531.623

### 3. Python, hand-written 

In [27]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7fe03d3f3b50>

In [28]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  6
  --------------
  minimum time:     617.443 ms (0.00% GC)
  median time:      620.353 ms (0.00% GC)
  mean time:        620.723 ms (0.00% GC)
  maximum time:     625.713 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1

In [29]:
sum_py(a)

4.999269608767693e6

In [30]:
sum_py(a) ≈ sum(a)

true

In [31]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "Python numpy"        => 2.72977
  "Python hand-written" => 617.443
  "Python built-in"     => 531.623

# 4. Julia (built-in) 

## Written directly in Julia, not in C!

In [32]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:889

In [33]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.072 ms (0.00% GC)
  median time:      2.228 ms (0.00% GC)
  mean time:        2.250 ms (0.00% GC)
  maximum time:     2.949 ms (0.00% GC)
  --------------
  samples:          2214
  evals/sample:     1

In [34]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "Python numpy"        => 2.72977
  "Python hand-written" => 617.443
  "Python built-in"     => 531.623
  "Julia built-in"      => 2.07188

# 5. Julia (hand-written) 

In [35]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [36]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.856 ms (0.00% GC)
  median time:      6.924 ms (0.00% GC)
  mean time:        6.934 ms (0.00% GC)
  maximum time:     7.256 ms (0.00% GC)
  --------------
  samples:          721
  evals/sample:     1

In [37]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "Python numpy"        => 2.72977
  "Julia hand-written"  => 6.8555
  "Python hand-written" => 617.443
  "Python built-in"     => 531.623
  "Julia built-in"      => 2.07188

# 6. Julia (hand-written w. simd) 

In [38]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [39]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.926 ms (0.00% GC)
  median time:      2.103 ms (0.00% GC)
  mean time:        2.135 ms (0.00% GC)
  maximum time:     3.161 ms (0.00% GC)
  --------------
  samples:          2333
  evals/sample:     1

In [40]:
mysum_simd(a)

4.999269608768217e6

In [41]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any, Any} with 6 entries:
  "Julia hand-written simd" => 1.92586
  "Python numpy"            => 2.72977
  "Julia hand-written"      => 6.8555
  "Python hand-written"     => 617.443
  "Python built-in"         => 531.623
  "Julia built-in"          => 2.07188

In [ ]:
?@simd

# Summary

In [42]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia hand-written simd.....1.9
Julia built-in..............2.1
Python numpy................2.7
Julia hand-written..........6.9
Python built-in...........531.6
Python hand-written.......617.4


# 7. The C language
C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

In [43]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [44]:
c_sum(a)

4.999269608767693e6

In [45]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [46]:
c_sum(a) - sum(a)

-5.047768354415894e-7

In [47]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.736 ms (0.00% GC)
  median time:      6.814 ms (0.00% GC)
  mean time:        6.861 ms (0.00% GC)
  maximum time:     8.252 ms (0.00% GC)
  --------------
  samples:          728
  evals/sample:     1

In [48]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 6.736285 msec


In [49]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any, Any} with 7 entries:
  "Julia hand-written simd" => 1.92586
  "Python numpy"            => 2.72977
  "Julia hand-written"      => 6.8555
  "C"                       => 6.73628
  "Python hand-written"     => 617.443
  "Python built-in"         => 531.623
  "Julia built-in"          => 2.07188

# 8. C with -ffast-math
If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [50]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum_fastmath (generic function with 1 method)

In [51]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.698 ms (0.00% GC)
  median time:      3.814 ms (0.00% GC)
  mean time:        3.849 ms (0.00% GC)
  maximum time:     5.692 ms (0.00% GC)
  --------------
  samples:          1297
  evals/sample:     1

In [52]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

3.697553

In [53]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia hand-written simd.....1.9
Julia built-in..............2.1
Python numpy................2.7
C -ffast-math...............3.7
C...........................6.7
Julia hand-written..........6.9
Python built-in...........531.6
Python hand-written.......617.4
